In [1]:
from libraries.utility import Utility as mutil
from libraries.db_base import db_base
from libraries.db_ins_fake_data import fake_data_to_db
from libraries.custom_excel_output import custom_excel_output
import pandas as pd
import pandasql as psql
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import warnings
import time
import os
import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))



In [2]:
dbprod = db_base("products")

Database : products, Connection Good: True


In [3]:
prod_info = dbprod.run_query_with_single_df(query_key="get_product_info")
prod_info.head(3)

,id,product_name,product_type
0,1,Nicholas's Patriotic Duty,Pay TV
1,2,Danielle's Date in the City,Pay TV
2,3,Haley's Big Romance,Pay TV


In [4]:
dbfin = db_base("finance",svr_type='postsql')

Database : finance, Connection Good: True


In [5]:
fin_account_activity = dbfin.run_query_with_single_df(query_key="get_account_activity")
fin_account_activity.head(3)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,5,1,777,$33.38,2023-06-30 23:14:29,Streaming Subscription,Hopkins-Campbell,Houston,Sell-Through Income Revenue 246506,Revenue
1,2,277,1,411,$6.62,2022-11-11 18:22:43,Home Video Rentals,Fernandez PLC,Houston,Interest Income Revenue 650062,Revenue
2,3,351,2,355,$23.12,2023-07-12 06:14:16,Music Publishing,"Mooney, Gomez and Frederick",Houston,Rental Income Revenue 677744,Revenue


In [6]:
dbcust = db_base("customers",svr_type='mysql')

Database : customers, Connection Good: True


In [7]:
cust_products = dbcust.run_query_with_single_df(query_key="get_customer_product_history1")
cust_products.head(3)

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,1,Craig,Smith,130,2023-09-25 10:44:30,2023-11-24 12:53:05,2023-11-08 04:48:06
1,1,Craig,Smith,5,2023-07-01 09:39:52,2023-09-14 23:14:29,2023-11-08 04:48:06
2,1,Craig,Smith,5,2023-06-30 23:14:29,2023-07-01 09:39:52,2023-11-08 04:48:06


In [8]:
sql = '''

select * from cust_products where id = 3

'''
df = psql.sqldf(sql)
df.head()

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,3,Mia,Callahan,468,2023-06-22 04:10:07.000000,2023-07-18 10:54:42.000000,2023-11-08 04:48:06.000000
1,3,Mia,Callahan,468,2023-06-21 10:54:42.000000,2023-06-22 04:10:07.000000,2023-11-08 04:48:06.000000
2,3,Mia,Callahan,288,2023-04-27 19:45:19.000000,2023-04-28 17:32:32.000000,2023-11-08 04:48:06.000000
3,3,Mia,Callahan,4,2022-12-17 21:26:52.000000,2023-01-13 05:29:53.000000,2023-11-08 04:48:06.000000
4,3,Mia,Callahan,4,2022-12-16 05:29:53.000000,2022-12-17 21:26:52.000000,2023-11-08 04:48:06.000000


In [9]:
fin_account_activity.head(1)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,5,1,777,$33.38,2023-06-30 23:14:29,Streaming Subscription,Hopkins-Campbell,Houston,Sell-Through Income Revenue 246506,Revenue


In [10]:
sql = '''
select
distinct
    cp.f_name,
    cp.l_name,
    p.product_name,
    p.product_type,
    fa.amt_usd,
    fa.post_date,
    fa.channel_desc,
    fa.partner_desc,
    fa.location_name,
    fa.account_name,
    fa.account_type
from cust_products as cp
join prod_info as p
on 
    p.id = cp.product_id
join fin_account_activity fa
on 
 fa.customer_id = cp.id
 and
 fa.product_id = cp.product_id
'''
df = psql.sqldf(sql)


In [11]:
dbcust = custom_excel_output(current_database="customers",svr_type='mysql')

Database : customers, Connection Good: True


In [15]:
# Clean up the post_date column
df['post_date'] = pd.to_datetime(df['post_date'])
df['post_date'] = df['post_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

u = mutil()
u.mkdir(os.path.join(u.get_this_dir(),"project_data","tempdir"))

display(md("### Example of Joined Data"))

file_name = os.path.join(dbcust.get_this_dir(),"project_data","tempdir","customer_account_activity.xlsx")

l_dfs = []
l_dfs.append(df)
dbcust.write_excel_from_dfs(list_of_dfs=l_dfs,file_name=file_name)
href_tag = dbcust.get_embedded_href_tag_from_image_file(file_name)
dbcust.nukefile(file_name)
display(HTML(href_tag))
display(df.head(30).style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]))

### Example of Joined Data

,f_name,l_name,product_name,product_type,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,Craig,Smith,Robert's Date in the City,Download,$10.24,2023-09-25 10:44:30,Web Series Platforms,Butler Ltd,Houston,Distribution Income Revenue 570683,Revenue
1,Craig,Smith,Paul's Great War,Subscription,$33.38,2023-06-30 23:14:29,Streaming Subscription,Hopkins-Campbell,Houston,Sell-Through Income Revenue 246506,Revenue
2,Craig,Smith,Paul's Great War,Subscription,$33.38,2023-07-01 09:39:52,Theme Parks,"Riggs, Hartman and Anderson",Houston,Theatrical Income Revenue 583291,Revenue
3,Craig,Smith,Michael's Patriotic Duty,Game,$6.62,2022-11-11 18:22:43,Home Video Rentals,Fernandez PLC,Houston,Interest Income Revenue 650062,Revenue
4,Craig,Smith,Michael's Patriotic Duty,Game,$6.62,2022-11-12 19:05:58,Airports and Airlines,Castro-Powell,Houston,Theatrical Income Revenue 798730,Revenue
5,Heather,Carrillo,David's Wild Ride,TV Season,$23.12,2023-07-12 06:14:16,Music Publishing,"Mooney, Gomez and Frederick",Houston,Rental Income Revenue 677744,Revenue
6,Heather,Carrillo,David's Wild Ride,TV Season,$23.12,2023-07-13 20:23:38,Cable TV Networks,Garcia-Stone,Houston,Streaming Cash Revenue 639148,Revenue
7,Heather,Carrillo,Melissa's Crazy Comedy,Single View Movie,$17.65,2023-04-11 12:05:11,International Distribution,Greer Group,Houston,Rental Income Revenue 716089,Revenue
8,Heather,Carrillo,Jesus's Big Romance,Movie,$28.39,2022-11-16 18:16:52,Streaming Aggregators,Herrera Ltd,Houston,Streaming Cash Revenue 914697,Revenue
9,Heather,Carrillo,Jesus's Big Romance,Movie,$28.39,2022-11-17 18:52:39,Airlines,"Buck, Roach and Robbins",Houston,Theatrical Income Revenue 690085,Revenue
